In [3]:
import pymysql
import logging
logging.basicConfig(level=logging.INFO)

mysql_host = "142.132.134.21"
mysql_prot = 9030
mysql_user = "dw"
mysql_pd = "a64imrJwqKZc9jiU!*aCYi.xnh9*TVhhBmZi"
mysql_db = "dw"
connection = pymysql.connect(host=mysql_host, user=mysql_user, password=mysql_pd, db=mysql_db, port=mysql_prot, charset='utf8')
cursor = connection.cursor()
logging.info("connection created")


def getBloNumRance():
    maxBloNumsql = """select max(block_number) from dw.dwb_nft_transfer_detail_eth_hi;"""
    cursor.execute(maxBloNumsql)
    res = cursor.fetchone()
    maxBloNum = res[0]

    minBloNumSql="select max(block_number) from dw.dm_nft_mint_detail_eth_hi;"
    cursor.execute(minBloNumSql)
    minres = cursor.fetchone()
    minBloNum = minres[0]

    return minBloNum,maxBloNum



def runSql():
    # 获取多个ods表中的最小blockno
    blorange = getBloNumRance()
    curminBlo = blorange[0]
    curmaxBlo = blorange[1]
    logging.info(blorange)
    logging.info("maxBlockNum: " + str(curminBlo) +"  minBlonum："+str(curmaxBlo))

    otherTokensql = """
    set query_timeout=1800; 
    INSERT INTO dw.dm_nft_mint_detail_eth_hi
    select account_address,token_address,token_id
            ,ts,block_number,transaction_hash
            ,`value`,gas_value/n,cost/n
    from
    (
        select account_address,token_address,token_id
            ,ts,block_number,id as transaction_hash
            ,value,gas_value,cost,is_mint
            ,1.0*count(*) over(partition by id)/2 as n
        from dw.dwb_nft_transfer_detail_eth_hi
        where block_number > {start} and block_number <={end}
    ) as a
    where is_mint = 1
   """.format(start=curminBlo, end=curmaxBlo)
    logging.info(otherTokensql)
    cursor.execute(otherTokensql)

if __name__ == '__main__':
    runSql()

INFO:root:connection created


In [4]:
sql = """
    insert into dw.dm_acct_nft_metrics_eth_byday_di
    select coalesce(b.account_address,s.account_address,m.account_address) as account_address
        ,coalesce(b.token_address,s.token_address,m.token_address) as token_address
        ,coalesce(b.partition_date,s.partition_date,m.partition_date) as partition_date
        ,b.buy_eth_amt
        ,b.buy_token_cnt
        ,b.buy_token_num
        ,b.buy_txn_cnt
        ,s.sell_eth_amt
        ,s.sell_token_cnt
        ,s.sell_token_num
        ,s.sell_txn_cnt
        ,m.mint_cost_eth_amt
        ,m.mint_gas_eth_amt
        ,m.mint_token_cnt
        ,m.mint_token_num
    from
    (
        select to_address as account_address,aa.token_address,date(trade_time) as partition_date
            ,sum(buyer_pay_amount) as buy_eth_amt
            ,count(distinct token_ids) as buy_token_cnt
            ,sum(token_num) as buy_token_num
            ,count(distinct transaction_hash) as buy_txn_cnt
        from dw.dwb_nft_trade_eth_detail_hi as aa
        where trade_time >= DATE_SUB(now(),1)
        and trade_time < date(now())
        group by 1,2,3
    ) as b

    full outer join
    (
        select from_address as account_address,aa.token_address,date(trade_time) as partition_date
            ,sum(buyer_pay_amount) as sell_eth_amt
            ,count(distinct token_ids) as sell_token_cnt
            ,sum(token_num) as sell_token_num
            ,count(distinct transaction_hash) as sell_txn_cnt
        from dw.dwb_nft_trade_eth_detail_hi as aa
        where trade_time >= DATE_SUB(now(),1)
        and trade_time < date(now())
        group by 1,2,3
    ) as s
    on b.account_address = s.account_address
    and b.token_address = s.token_address
    and b.partition_date = s.partition_date

    full outer join
    (
        select account_address,token_address, date(mint_time) as partition_date
            ,sum(mint_cost) as mint_cost_eth_amt
            ,sum(gas_cost) as mint_gas_eth_amt
            ,count(distinct token_id) as mint_token_cnt
            ,sum(mint_amount) as mint_token_num
        from dw.dwb_nft_mint_detail_eth_hi
        where mint_time >= DATE_SUB(now(),1)
        and mint_time < date(now())
        group by 1,2,3
    ) as m
    on b.account_address = m.account_address
    and b.token_address = m.token_address
    and b.partition_date = m.partition_date
"""